In [ ]:
from tkinter import *
from tkinter import messagebox
from PIL import ImageTk, Image
from googletrans import Translator
import requests
import json
import os
import random
import cv2
import numpy as np
from sklearn.cluster import KMeans


def translate_city_name_to_en(city_name):
    translator = Translator()
    return translator.translate(city_name, src='ko', dest='en').text


def translate_weather_description_to_ko(weather_description):
    translator = Translator()
    return translator.translate(weather_description, src='en', dest='ko').text


def get_weather_by_city(city_name):
    api_key = '"OPENWEATHER_API_KEY"'  # OpenWeatherMap API 키
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = json.loads(response.text)

    if (data['cod'] == 200):
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        weather_description = data['weather'][0]['description']
        return temperature, humidity, translate_weather_description_to_ko(weather_description)
    else:
        return None, None, None


def calculate_distance(rgb1, rgb2):
    r1, g1, b1 = rgb1
    r2, g2, b2 = rgb2
    distance = np.sqrt((r2 - r1) ** 2 + (g2 - g1) ** 2 + (b2 - b1) ** 2)
    return distance


def get_color_name(rgb):
    color_names = {
        (0, 0, 0): "검정색",
        (255, 255, 255): "흰색",
        (255, 0, 0): "빨간색",
        (0, 255, 0): "초록색",
        (0, 0, 180): "파란색",
        (170, 170, 170): "회색",
        (0, 0, 80): "남색",
        (235, 235, 235): "베이지",
    }
    
    min_distance = float('inf')
    closest_color = None
    for color, name in color_names.items():
        distance = calculate_distance(rgb, color)
        if distance < min_distance:
            min_distance = distance
            closest_color = name

    return closest_color


def get_dominant_color(image, k=20):
    height, width, _ = image.shape
    center_x = width // 2
    center_y = height // 2

    region_size = min(center_x, center_y)
    region = image[center_y - region_size:center_y + region_size, center_x - region_size:center_x + region_size]
    region_rgb = cv2.cvtColor(region, cv2.COLOR_BGR2RGB)
    region_flattened = region_rgb.reshape((-1, 3))
    
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(region_flattened)
    cluster_centers = kmeans.cluster_centers_
    
    cluster_labels, cluster_counts = np.unique(labels, return_counts=True)
    sorted_clusters = cluster_labels[np.argsort(-cluster_counts)]

    dominant_colors_rgb = cluster_centers[sorted_clusters[:k]].astype(int)
    
    color_names = []
    for color_rgb in dominant_colors_rgb:
        color_name = get_color_name(color_rgb)
        color_names.append(color_name)
    
    return dominant_colors_rgb, color_names


def show_weather_info():
    city_name = city_name_var.get()
    if city_name == '':
        return

    translated_city_name = translate_city_name_to_en(city_name)
    temperature, humidity, weather_description = get_weather_by_city(translated_city_name)
    if temperature is not None and humidity is not None and weather_description is not None:
        weather_description_var.set(weather_description)
        temperature_var.set(f'{temperature} °C')
        humidity_var.set(f'{humidity} %')
        show_clothing_recommendation(temperature)
    else:
        messagebox.showwarning('오늘의 코디', '날씨 정보를 가져오는데 실패했습니다.')


def show_clothing_recommendation(temperature):
    image_path = r'C:\날씨별옷코디\project11'
    short_pair = [('beize_short_t-removebg-preview.png', 'beize_short_p-removebg-preview.png'),
                ('black_short_t-removebg-preview.png', 'black_short_p-removebg-preview.png'),
                ('blue_short_t-removebg-preview.png', 'blue_short_p-removebg-preview.png'),
                ('gray_short_t-removebg-preview.png', 'gray_short_p-removebg-preview.png')]
    long_pair =  [('white_long_t-removebg-preview.png', 'beize_long_p-removebg-preview.png'),
                ('black_long_t-removebg-preview.png', 'black_long_p-removebg-preview.png'),
                ('gray_long_t-removebg-preview.png', 'gray_long_p-removebg-preview.png'),
                ('navy_long_t-removebg-preview.png', 'navy_long_p-removebg-preview.png')]
    
    if int(temperature) <= 20:
        t, p = random.choice(long_pair)
    else:
        t, p = random.choice(short_pair)

    image_t = Image.open(os.path.join(image_path, t))
    image_p = Image.open(os.path.join(image_path, p))
    
    base_height = 200
    width_t = int(image_t.size[0] * (base_height / image_t.size[1]))
    image_t = image_t.resize((width_t, base_height), Image.ANTIALIAS)

    width_p = int(image_p.size[0] * (base_height / image_p.size[1]))
    image_p = image_p.resize((width_p, base_height), Image.ANTIALIAS)

    image_t_tk = ImageTk.PhotoImage(image_t)
    image_p_tk = ImageTk.PhotoImage(image_p)

    t_image_label['image'] = image_t_tk
    t_image_label['width'] = width_t
    t_image_label['height'] = base_height
    t_image_label.image = image_t_tk

    p_image_label['image'] = image_p_tk
    p_image_label['width'] = width_p
    p_image_label['height'] = base_height
    p_image_label.image = image_p_tk

    # Show dominant color
    image_t_cv2 = cv2.imread(os.path.join(image_path, t))
    image_p_cv2 = cv2.imread(os.path.join(image_path, p))
    dominant_colors_t, color_names_t = get_dominant_color(image_t_cv2)
    dominant_colors_p, color_names_p = get_dominant_color(image_p_cv2)
    
    print(f"Top Dominant Color: {color_names_t[0]}")
    print(f"Bottom Dominant Color: {color_names_p[0]}")


root = Tk()
root.title('오늘의 코디')

city_name_var = StringVar()
top_frame = Frame(root)
Label(top_frame, text='도시 이름(한글): ').grid(row=0, column=0)
Entry(top_frame, textvariable=city_name_var).grid(row=0, column=1)
Button(top_frame, text=' 검색 ', command=show_weather_info).grid(row=0, column=2, padx=5)
top_frame.grid(row=0, column=0)

weather_description_var = StringVar()
temperature_var = StringVar()
humidity_var = StringVar()
middle_frame = LabelFrame(root, text='날씨정보')
Label(middle_frame, text='시정: ').grid(row=0, column=0)
Label(middle_frame, text='온도: ').grid(row=1, column=0)
Label(middle_frame, text='습도: ').grid(row=2, column=0)
Label(middle_frame, textvariable=weather_description_var).grid(row=0, column=1)
Label(middle_frame, textvariable=temperature_var).grid(row=1, column=1)
Label(middle_frame, textvariable=humidity_var).grid(row=2, column=1)
middle_frame.grid(row=1, column=0)

bottom_frame = LabelFrame(root, text='오늘의 코디')
t_image_label = Label(bottom_frame, width=40, height=10)
t_image_label.pack(fill='both')
p_image_label = Label(bottom_frame, width=40, height=10)
p_image_label.pack(fill='both')
bottom_frame.grid(row=2, column=0)

for child in root.winfo_children():
    child.grid_configure(padx=10, pady=10, sticky='ew')

mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\test\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\LHE\AppData\Local\Temp/ipykernel_14268/3965570935.py", line 107, in show_weather_info
    show_clothing_recommendation(temperature)
  File "C:\Users\LHE\AppData\Local\Temp/ipykernel_14268/3965570935.py", line 154, in show_clothing_recommendation
    dominant_colors_t, color_names_t = get_dominant_color(image_t_cv2)
  File "C:\Users\LHE\AppData\Local\Temp/ipykernel_14268/3965570935.py", line 70, in get_dominant_color
    height, width, _ = image.shape
AttributeError: 'NoneType' object has no attribute 'shape'
